## Import Dependencies

In [1]:
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta


## Import CSV and Pre-Processing

In [2]:
source = pd.read_csv('Trades.csv')

#source data pre-processing


source['TradeDateTime'] = pd.to_datetime(source['TradeDateTime'])
source.sort_values(by='TradeDateTime',ascending=False)


arr_date = source['TradeDateTime'].dt.to_pydatetime()
source['TradeDateTime'] = pd.Series(arr_date, dtype=object)

#Converts Emission Venue A & B into just Emission for querying

source['Product'] = np.where(source['Product'].str.contains('Emission', na=False), "Emission", source['Product'])

source = source.set_index('TradeDateTime').sort_index()


## Supporting Functions

Here I've defined the supporting subroutines to help assist my main function and to make the code more readable, we're utilsing datetime for its tools in time manipulation which is key to this solution.

In [3]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta




#'Apr 18 2022 1:45PM' - acceptable format for begin and end variables.




def dt_manipulation(datetime,freq,begin,end):

    if freq == '15MIN':
        td = timedelta(minutes=15)

    elif freq =='1H':
        td = timedelta(hours=1)
        begin = begin.replace(minute = 0)
        end = end.replace(minute = 0)

    elif freq =='1D':
        td = timedelta(days=1) 
        begin = begin.replace(hour = 7,minute = 0)
        end = end.replace(hour = 17,minute = 0)   

    dt_list = [dt for dt in 
       datetime_range(begin, end, 
       td)]
    


    if freq != 'D':# a bit messy but due to time constraints
        dt_series = pd.DatetimeIndex(dt_list)
        
        dt_series = pd.Series(data = dt_list, index = dt_series)
        dt_series = dt_series.between_time('07:00', '17:00')

    else:
        dt_series = pd.Series(dt_list)

    try:
        last_dt = dt_series[dt_series <=datetime].iloc[-1]
    except:
        last_dt = dt_series[0]

    return last_dt

## Main Function

This Subroutine utilises many techniques to manipulate the data into the requested format, it does so using apply, iteration alongside its support functions created earlier.

In [4]:
def create_ohlc_vol_df(begin,end,freq,products = []):

    begin = datetime.strptime(begin, '%b %d %Y %I:%M%p')
    end = datetime.strptime(end, '%b %d %Y %I:%M%p')

    for x, product in enumerate( products): # converts emission products with different venues into just Emission as a singular product

        if "Emission" in product:
            products[x] == "Emission"
    
    products = list(dict.fromkeys(products)) # Removes dupes as a result

    universe = source[source['Product'].isin(products)]

    universe = universe.loc[begin:end]

    #creating version of universe for only within trading hours

    


    #create required columns - label columns indicating the timeframe which can be used to pivot

    output = {}

    if freq == '1D':
        universe['1D'] = universe.index.to_series().apply(lambda x: dt_manipulation(x,'1D',begin,end))

        for x, contract in enumerate(universe['Contract'].unique()):
            
            output[contract] =pd.pivot_table(universe[universe['Contract']==contract],columns=['Product'], values=['Price','Quantity'], index=['1D'],
                            aggfunc={'Price':[min,max,'first','last'],'Quantity':np.sum})






    elif freq == '15MIN':
        universe_trade_hours = universe.between_time('07:00', '17:00')

        universe_trade_hours['15MIN'] = universe_trade_hours.index.to_series().apply(lambda x: dt_manipulation(x,'15MIN',begin,end) )


        for x, contract in enumerate(universe['Contract'].unique()):
        
            output[contract] =pd.pivot_table(universe_trade_hours[universe_trade_hours['Contract']==contract],columns=['Product'], values=['Price','Quantity'], index=['15MIN'],
                            aggfunc={'Price':[min,max,'first','last'],'Quantity':np.sum})




    elif freq == '1H':
        universe_trade_hours = universe.between_time('07:00', '17:00')

        universe_trade_hours['1H'] = universe_trade_hours.index.to_series().apply(lambda x: dt_manipulation(x,'1H',begin,end) )

        for x, contract in enumerate(universe['Contract'].unique()):
        
            output[contract] =pd.pivot_table(universe_trade_hours[universe_trade_hours['Contract']==contract],columns=['Product'], values=['Price','Quantity'], index=['1H'],
                            aggfunc={'Price':[min,max,'first','last'],'Quantity':np.sum})





    return output

# Testing

Please note that first, last, max and min are open,close,high and low respectively alongside Quantity sum which is trading volumne

The output is a dictionary with multiple datasets, the keys correspond to which contract the underlying dataframe represents.

In [5]:
create_ohlc_vol_df('Apr 18 2022 1:45PM','Apr 19 2022 7:00AM','15MIN',['Energy'])


{'Q01':                        Price                            Quantity
                        first     last      max      min      sum
 Product               Energy   Energy   Energy   Energy   Energy
 15MIN                                                           
 2022-04-18 13:45:00  23.9175  23.9625  23.9950  23.9050      212
 2022-04-18 14:00:00  23.9525  24.0175  24.1125  23.9525      467
 2022-04-18 14:15:00  24.0025  24.0375  24.0400  23.9250      233
 2022-04-18 14:30:00  24.0500  23.9900  24.0975  23.9775      481
 2022-04-18 14:45:00  23.9975  24.0450  24.1350  23.9925      493
 2022-04-18 15:00:00  24.0450  24.0250  24.0750  24.0175      334
 2022-04-18 15:15:00  24.0175  24.0850  24.0900  24.0175      228
 2022-04-18 15:30:00  24.0825  24.0850  24.1300  24.0725      352
 2022-04-18 15:45:00  24.0850  24.0975  24.1450  24.0850      373
 2022-04-18 16:00:00  24.0975  24.1550  24.2000  24.0975      854
 2022-04-18 16:15:00  24.1500  24.0225  24.1500  23.9775      868
 20

In [6]:
create_ohlc_vol_df('Apr 18 2022 1:45PM','Apr 19 2022 10:45AM','1D',['Energy','Emission'])

{'Q01':                        Price                           Quantity
                        first     last      max     min      sum
 Product               Energy   Energy   Energy  Energy   Energy
 1D                                                             
 2022-04-18 07:00:00  23.9175  24.0725  24.2000  23.905     5775
 2022-04-19 07:00:00  24.2550  24.4375  24.6225  24.250    10038,
 'DA':                        Price                              Quantity
                        first      last       max      min      sum
 Product             Emission  Emission  Emission Emission Emission
 1D                                                                
 2022-04-18 07:00:00  19.6125  20.13125  20.20625  19.3125    72056
 2022-04-19 07:00:00  20.1750  20.29375  20.50000  20.1750    22152,
 'M01':                        Price                              Quantity
                        first      last      max       min      sum
 Product             Emission  Emission Emis